Start scheduler

In [1]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

import dask_cudf

import cudf

import numpy as np


from dask_cuml import knn as cumlKNN

client = Client(scheduler_file = "cluster.json")


In [2]:
X = cudf.DataFrame([('a', np.array([0, 1, 2, 3, 4], np.float32)),
                    ('b', np.array([5, 6, 7, 7, 8], np.float32))])

X_df = dask_cudf.from_cudf(X, chunksize=1).persist()

In [3]:
lr = cumlKNN.KNN()
lr.fit(X_df)

KEY TO PART DICT: {"('from_cudf-8d8d8925750949c58004971c690f39cd', 0)": <Future: status: finished, type: DataFrame, key: ('from_cudf-8d8d8925750949c58004971c690f39cd', 0)>, "('from_cudf-8d8d8925750949c58004971c690f39cd', 1)": <Future: status: finished, type: DataFrame, key: ('from_cudf-8d8d8925750949c58004971c690f39cd', 1)>, "('from_cudf-8d8d8925750949c58004971c690f39cd', 2)": <Future: status: finished, type: DataFrame, key: ('from_cudf-8d8d8925750949c58004971c690f39cd', 2)>, "('from_cudf-8d8d8925750949c58004971c690f39cd', 3)": <Future: status: finished, type: DataFrame, key: ('from_cudf-8d8d8925750949c58004971c690f39cd', 3)>}
WHO HAS: {"('from_cudf-8d8d8925750949c58004971c690f39cd', 2)": ('tcp://10.2.172.107:36488',), "('from_cudf-8d8d8925750949c58004971c690f39cd', 1)": ('tcp://10.2.172.107:41402',), "('from_cudf-8d8d8925750949c58004971c690f39cd', 3)": ('tcp://10.2.172.107:41402',), "('from_cudf-8d8d8925750949c58004971c690f39cd', 0)": ('tcp://10.2.172.107:36488',)}
WORKER_MAP: [(('10.

In [4]:
g = lr.kneighbors(X, 4)
